## Working with Custom Models

### https://datarobot-public-api-client.readthedocs-hosted.com/en/v2.23.0/entities/custom_model.html

In [14]:
import pandas as pd
import numpy as np
import os
import datarobot as dr

In [4]:
USERNAME = os.environ['DATAROBOT_USERNAME']
API_TOKEN = os.environ['DATAROBOT_API_TOKEN']
ENDPOINT = 'https://app.datarobot.com/api/v2'

dr.Client(token=API_TOKEN, endpoint=ENDPOINT)  

In [20]:
# List custom models and grab one
cmlist = dr.CustomInferenceModel.list()
cm = np.nan
for i, model in enumerate(cmlist):
    if 'Housing price regression' in cmlist[i].name:
        cm = cmlist[i]
        break
print(cm.id)
cm

5fbe920c59e8fd54c8556adb


CustomInferenceModel('Housing price regression')

In [21]:
# # Download latest content
# path_to_download = '/Users/matthew.cohen/Dev/data-science-scripts/matt'
# custom_model = dr.CustomInferenceModel.get(cm.id)
# custom_model.download_latest_version(path_to_download)

In [43]:
# Create a new custom model
# dr.CustomInferenceModel.create?
cm_new = dr.CustomInferenceModel.create(
    name='boston housing regr api',
    target_type=dr.TARGET_TYPE.REGRESSION,
    target_name='MEDV',
    language='Python',
    description='test from api call',
    network_egress_policy=None,
)

In [44]:
# Assign training data, if it hasn't been uploaded already
path_to_dataset = '/Users/matthew.cohen/Documents/DR/MLOps/_custom_models_and_env/data/boston_housing_train.csv'

try:
    dr.Dataset.get('601af9df93178624eaaa1761')
    print('custom model dataset already exists')
except:
    print('dataset not found, creating one')
    dataset = dr.Dataset.create_from_file(file_path=path_to_dataset)

cm_new.assign_training_data(dataset.id)
cm_new.training_data_file_name

custom model dataset already exists


Dataset(name='boston_housing_train.csv', id='601af9df93178624eaaa1761')

In [52]:
# Add the model files
# Modeling code for Custom Inference Models can be uploaded by creating a Custom Model Version
custom_model_folder = "/Users/matthew.cohen/Documents/DR/MLOps/_custom_models_and_env/Housing_prices"

# Get and use the sklearn drop-in env
ees = dr.ExecutionEnvironment.list()
for i, ee in enumerate(ees):
    if 'Scikit-Learn Drop-In' in ees[i].name:
        ee = ees[i]
print('Get the %s env' % ee.name)

# add files from the folder to the custom model
model_version = dr.CustomModelVersion.create_clean(
    custom_model_id=cm_new.id,
    base_environment_id=ee.id,
    folder_path=custom_model_folder,
)
model_version

# # or add a list of files to the custom model
# model_version_2 = dr.CustomModelVersion.create_clean(
#     custom_model_id=cm_new.id,
#     files=[(os.path.join(custom_model_folder, 'custom.py'), 'custom.py')],
# )

# # and/or set k8s resources to the custom model
# model_version_3 = dr.CustomModelVersion.create_clean(
#     custom_model_id=custom_model.id,
#     files=[(os.path.join(custom_model_folder, 'custom.py'), 'custom.py')],
#     network_egress_policy=dr.NETWORK_EGRESS_POLICY.PUBLIC,
#     desired_memory=256*1024*1024,
#     maximum_memory=512*1024*1024,
#     replicas=1,
# )

Get the [DataRobot] Python 3 Keras Drop-In env


CustomModelVersion('v1.0')

In [51]:
ees = dr.ExecutionEnvironment.list()
for i, ee in enumerate(ees):
    if 'Scikit-Learn Drop-In' in ees[i].name:
        ee = ees[i]
ee

ExecutionEnvironment('[DataRobot] Python 3 Keras Drop-In')

In [56]:
dr.models.FeatureDrift
from datarobot.models import FeatureDrift